<a href="https://colab.research.google.com/github/romazzo89/romazzo89/blob/main/Trabajo-Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
